# Skyline Optimization Project

### Importing Libraries

In [6]:
import gurobipy as gb
from gurobipy import *
import pandas as pd
import os

### Importing Data

In [7]:
os.chdir("/Users/lucaspenney/Documents/McGill/MGSC 662/Skyline Group Project")
beverage_sales = pd.read_excel("Beverage Sales - Nov 2024 - Oct 2025.xlsx", sheet_name = None)
wine_costing = pd.read_excel("Beverage Costing.xlsx")

In [8]:
wine_costing.head()

,Item,Glass ($),COGS,Bottle,COGS.1,COST,LEAD TIME\n(DAYS)
0,WINE,NaN,NaN,NaN,NaN,NaN,NaN
1,Champagne (G - 120ml),NaN,NaN,NaN,NaN,NaN,NaN
2,"Veuve Clicquot Rosé, Reims, France",NaN,NaN,198.0,0.617573,106.33,3.0
3,"Veuve Clicquot, Reims, France",NaN,NaN,156.0,0.475186,64.46,3.0
4,"Moët Chandon, Épernay, France",23.0,0.44968,124.0,0.521302,56.21,3.0


In [9]:
#Viewing what the first dataframe looks like
print(beverage_sales['Dec-24'])

     LOCATION_ID         LOCATION  PRODUCT_ID             CATEGORY  \
0              8  Buffet Rest/Bar       22918  BEVA - Wine | White   
1              8  Buffet Rest/Bar       26267  BEVA - Wine | White   
2              8  Buffet Rest/Bar       22916  BEVA - Wine | White   
3              8  Buffet Rest/Bar       30900  BEVA - Wine | White   
4              8  Buffet Rest/Bar       14663  BEVA - Wine | White   
..           ...              ...         ...                  ...   
333           10             Cafe         423       BEVA - Spirits   
334           10             Cafe       21401       BEVA - Spirits   
335           10             Cafe       31528       BEVA - Spirits   
336           10             Cafe         453       BEVA - Spirits   
337           10             Cafe         436       BEVA - Spirits   

           ATTRIBUTE                          PRODUCT  DISPLAY_ORDER  \
0    Sauvignon Blanc              Cloudy Bay SB 750ml              5   
1    Sauvignon 

### Combining the Monthly Sales Data into a Single Dataframe
- The data was provided as an Excel file, where each tab was a different month and included sales of all beverages, beyond wine. Within each month, a wine could appear more than once if it was sold in more than one location (eg. cafe, restaurant, market kitchen)
- First, we filtered to only include wine, cleaned up the product names and created a long format dataframe with all the wines sold, quantities and their respective months
- Then, we pivoted the monthly data to wide format, creating a dataframe with the months as column, wines as rows and the entries being the sales
- Columns were added for sales over the 11 month period we had data for, as well as being extrapolated linearly to 12 months of sales

In [40]:
# Placeholder for the column that identifies the 75 types of wine
WINE_IDENTIFIER_COLUMN = "PRODUCT"

all_months_data = []
target_category = 'BEVA - Wine'

#Creates a long format dataframe that combines all the monthly sales data, creating a 'Product' 'Category', 'Quantity' and 'Month' column
for month, df in beverage_sales.items():
    # 1. Filter: Keep only rows where CATEGORY contains 'BEVA - WINE'
    df_filtered = df[
        df['CATEGORY'].str.contains(
            target_category, 
            case=False,  # Set to False to ignore case (e.g., 'Beva - Wine' will match 'BEVA - WINE')
            na=False     # Set to False to treat NaN values as non-matches (prevents errors)
        )
    ].copy()
    
    #Remove asterisks from the product column, whitespace
    df_filtered['PRODUCT'] = df_filtered['PRODUCT'].astype(str).str.replace('*', '', regex=False)
    df_filtered['PRODUCT'] = df_filtered['PRODUCT'].str.strip()

    # 2. Select: Keep only the essential columns (Wine ID, Category, Quantity)
    # Keep 'CATEGORY' for validation, and the identifier and quantity for the final output.
    df_selected = df_filtered[[WINE_IDENTIFIER_COLUMN, 'CATEGORY', 'QUANTITY']]
    

    df_aggregated = df_selected.groupby(WINE_IDENTIFIER_COLUMN, as_index=False)['QUANTITY'].sum()
    
    # 3. Add the 'Month' column
    df_aggregated['Month'] = month
    
    # 4. Add the processed DataFrame to the list
    all_months_data.append(df_aggregated)

# Concatenate all monthly DataFrames into one large DataFrame
combined_df = pd.concat(all_months_data, ignore_index=True)

#Display the number of unique wines they sold across all months
total_unique_wines = combined_df[WINE_IDENTIFIER_COLUMN].nunique()
print(f"Total unique wines across all months: {total_unique_wines}\n")

# Print a preview of the combined, filtered data
print("--- Preview of Combined, Filtered Data ---")
print(combined_df.head())
print(f"\nTotal rows in long format: {len(combined_df)}\n")

# Pivot the Data  from long format to wide format

final_df = combined_df.pivot_table(
    index=WINE_IDENTIFIER_COLUMN,  # The 75 wine types become the rows
    columns='Month',               # The months become the columns
    values='QUANTITY',             # The sales figures are the cell values
    aggfunc='sum'                  # This sums duplicates (e.g., 2 sales of 'Cabernet' in Jan)
).reset_index()



#Dropping the 'Year' column, since it doesnt acccurately reflect sales over the period
final_df = final_df.drop(columns=['Year'])

# Handle NaNs (if some wines weren't sold in a month)
# By default, pivot_table will leave NaN. You can replace them with 0.
final_df = final_df.fillna(0)

#Fixes the order of the columns
month_order = [
    'Dec-24', 'Jan-25', 'Feb-25', 'Mar-25', 'Apr-25', 'May-25',
    'Jun-25', 'Jul-25', 'Aug-25', 'Sep-25', 'Oct-25'
]

# 2. Re-order the DataFrame columns
# This keeps the PRODUCT column and adds the months in the correct order
final_df = final_df[[WINE_IDENTIFIER_COLUMN] + month_order]

#Removes name of index column
final_df.columns.name = None

#Identifies all columns that arent wine names as those to be summed
month_columns = [col for col in final_df.columns if col != WINE_IDENTIFIER_COLUMN]

# Creates a new 'TotalSales_11Months' column, since we have 11 months of data
final_df['TotalSales_11Months'] = final_df[month_columns].sum(axis=1)

# Define a scaling factor to convert 11 months of data to annual
scaling_factor = 12 / 11

# Create 'AdjustedAnnualSales' column, round to whole number
final_df['AdjustedAnnualSales'] = final_df['TotalSales_11Months'] * scaling_factor
final_df['AdjustedAnnualSales'] = final_df['AdjustedAnnualSales'].round(0)

# Display the final, clean DataFrame
print("--- Combined and Filtered Wide Format Dataframe ---")
print(final_df.head())
print(f"\nTotal rows in wide format: {len(final_df)}\n")

Total unique wines across all months: 148

--- Preview of Combined, Filtered Data ---
                           PRODUCT  QUANTITY   Month
0        Anniversary Bubbles 150ml        61  Dec-24
1           Ant Moore Chardy 150ml        65  Dec-24
2           Ant Moore Chardy 750ml         2  Dec-24
3               Ant Moore SB 150ml        33  Dec-24
4  Ant Moore Sauvignon Blanc 750ml         1  Dec-24

Total rows in long format: 1120

--- Combined and Filtered Wide Format Dataframe ---
                           PRODUCT  Dec-24  Jan-25  Feb-25  Mar-25  Apr-25  \
0        Anniversary Bubbles 150ml    61.0    88.0    85.0    85.0    58.0   
1           Ant Moore Chardy 150ml    65.0    87.0    86.0   102.0    56.0   
2           Ant Moore Chardy 750ml     2.0     1.0     4.0     3.0     1.0   
3               Ant Moore SB 150ml    33.0    69.0    59.0    84.0    54.0   
4  Ant Moore Sauvignon Blanc 750ml     1.0     6.0     2.0     9.0     4.0   

   May-25  Jun-25  Jul-25  Aug-25  Sep-25

In [42]:
# To confirm the data was correctly translated from the Excel file 
# Checks that the Rabbit Ranch wine sales is equal to 40
# The Rabbit Ranch SB 750ml wine was sold in both the cafe and the restaurant in October 2025, so this confirms these two entries were combined appropriately and are in the right row/column
product_row = final_df.query("PRODUCT == 'Rabbit Ranch SB 750ml'")
print(product_row["Oct-25"].iloc[0])

40.0
